In [1]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct.git
!ls pubmed-rct

Cloning into 'pubmed-rct'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 39 (delta 8), reused 5 (delta 5), pack-reused 25
Receiving objects: 100% (39/39), 177.08 MiB | 35.39 MiB/s, done.
Resolving deltas: 100% (15/15), done.
Updating files: 100% (13/13), done.
PubMed_200k_RCT				       PubMed_20k_RCT_numbers_replaced_with_at_sign
PubMed_200k_RCT_numbers_replaced_with_at_sign  README.md
PubMed_20k_RCT


In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch import nn
import torchtext

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

batch_size = 32

max_length = 128

cuda


In [4]:
!mkdir /content/modules
!wget -P /content/modules https://raw.githubusercontent.com/mmehmetaliyavuz/SkimLit/main/dataframe.py
!wget -P /content/modules https://raw.githubusercontent.com/mmehmetaliyavuz/SkimLit/main/dftonlp.py

--2024-04-10 10:24:54--  https://raw.githubusercontent.com/mmehmetaliyavuz/SkimLit/main/dataframe.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1950 (1.9K) [text/plain]
Saving to: ‘/content/modules/dataframe.py’

dataframe.py        100%[===================>]   1.90K  --.-KB/s    in 0s      

2024-04-10 10:24:54 (39.7 MB/s) - ‘/content/modules/dataframe.py’ saved [1950/1950]

--2024-04-10 10:24:54--  https://raw.githubusercontent.com/mmehmetaliyavuz/SkimLit/main/dftonlp.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3473 

In [5]:
from modules.dataframe import dfPubMed
test_path = "/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/test.txt"
train_path = "/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/train.txt"
dataframePubMed = dfPubMed(train_path,test_path)
train_data = dataframePubMed.get_dataframe(train_path)
test_data = dataframePubMed.get_dataframe(test_path)

In [6]:
train_data.head()

,id,type,sentence,order,length
0,24293578,OBJECTIVE,To investigate the efficacy of @ weeks of dail...,1,12
1,24293578,METHODS,A total of @ patients with primary knee OA wer...,2,12
2,24293578,METHODS,Outcome measures included pain reduction and i...,3,12
3,24293578,METHODS,Pain was assessed using the visual analog pain...,4,12
4,24293578,METHODS,Secondary outcome measures included the Wester...,5,12


In [7]:
from modules.dftonlp import nlpPudMed
nlp_processor = nlpPudMed(train_data, test_data)
padded_train,padded_test = nlp_processor.preprocess_data()
train_labels,test_labels = nlp_processor.encode_labels()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['i', 'me', 'my', 'myself', 'we']


/usr/local/lib/python3.10/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


In [8]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class CustomDataset(Dataset):
    def __init__(self, datas, labels):
        self.data = datas
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = self.data[idx]
        label = self.labels[idx]
        return sentence, label

In [9]:
import os
num_cpus = os.cpu_count()
num_cpus
train_loader = CustomDataset(padded_train,train_labels)
train_dataloader = DataLoader(dataset=train_loader, batch_size=32, shuffle=True, pin_memory=True, num_workers=num_cpus)
test_loader = CustomDataset(padded_test,test_labels)
test_dataloader = DataLoader(dataset=test_loader, batch_size=32, shuffle=True, pin_memory=True, num_workers=num_cpus)

In [10]:
all_data = torch.cat([data for data, _ in train_dataloader], dim=0)
unique_elements = torch.unique(all_data)
VOCAB_SIZE = len(unique_elements)


In [11]:
all_data = torch.cat([label for _, label in train_dataloader], dim=0)
unique_elements = torch.unique(all_data)
NUM_CLASSES = len(unique_elements)
NUM_CLASSES

5

In [12]:
EMBEDDING_DIM = 128


In [13]:
embed = torch.nn.Embedding(VOCAB_SIZE,128)

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TextCNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes, kernel_size=5, num_filters=64):
        super(TextCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv = nn.Conv1d(embedding_dim, num_filters, kernel_size)
        self.global_pooling = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(num_filters, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)
        embedded = embedded.permute(0, 2, 1)
        conv_out = self.conv(embedded)
        activated = F.relu(conv_out)
        pooled = self.global_pooling(activated).squeeze(dim=-1)
        output = self.fc(pooled)
        return F.softmax(output, dim=-1)



model_1 = TextCNN(VOCAB_SIZE, EMBEDDING_DIM, NUM_CLASSES)


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TextCNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes, kernel_size=5, num_filters=64):
        super(TextCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv = nn.Conv1d(embedding_dim, num_filters, kernel_size)
        self.global_pooling = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(num_filters, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)  # [batch_size, seq_len, embedding_dim]
        embedded = embedded.permute(0, 2, 1)  # [batch_size, embedding_dim, seq_len]
        conv_out = self.conv(embedded)  # [batch_size, num_filters, seq_len - kernel_size + 1]
        activated = F.relu(conv_out)
        pooled = self.global_pooling(activated).squeeze(dim=-1)  # [batch_size, num_filters]
        output = self.fc(pooled)  # [batch_size, num_classes]
        return F.softmax(output, dim=-1)


model = TextCNN(VOCAB_SIZE, EMBEDDING_DIM, NUM_CLASSES)
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)


In [17]:
from tqdm import tqdm
def train_model(model, train_loader, val_loader, loss_fn, optimizer, scheduler, num_epochs):
    for epoch in tqdm(range(num_epochs)):
        print(f"Epoch: {epoch}\n-------")
        ### Training
        train_loss = 0
        correct = 0
        total = 0
        # Add a loop to loop through training batches
        for batch, (X, y) in enumerate(train_loader):
            model.train()
            # 1. Forward pass
            y_pred = model(X)

            # 2. Calculate loss (per batch)
            loss = loss_fn(y_pred, y)
            train_loss += loss.item() # accumulatively add up the loss per epoch

            # 3. Optimizer zero grad
            optimizer.zero_grad()

            # 4. Loss backward
            loss.backward()

            # 5. Optimizer step
            optimizer.step()

            # Print out how many samples have been seen
            if batch % 400 == 0:
                print(f"Looked at {batch * len(X)}/{len(train_loader.dataset)} samples")

            _, predicted = torch.max(y_pred, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()

        # Compute training loss and accuracy
        train_loss /= len(train_loader.dataset)
        train_accuracy = correct / total

        # Validation loop
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            # Compute validation loss and accuracy
            val_loss /= len(val_loader.dataset)
            val_accuracy = correct / total

        print(f"Epoch {epoch + 1}/{num_epochs}, "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

        # Update the learning rate scheduler
        scheduler.step(val_loss)

num_epochs = 3
train_model(model, train_dataloader, test_dataloader, loss_fn, optimizer, scheduler, num_epochs)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
-------


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Looked at 0/180040 samples
Looked at 12800/180040 samples
Looked at 25600/180040 samples
Looked at 38400/180040 samples
Looked at 51200/180040 samples
Looked at 64000/180040 samples
Looked at 76800/180040 samples
Looked at 89600/180040 samples
Looked at 102400/180040 samples
Looked at 115200/180040 samples
Looked at 128000/180040 samples
Looked at 140800/180040 samples
Looked at 153600/180040 samples
Looked at 166400/180040 samples
Looked at 179200/180040 samples


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
 33%|███▎      | 1/3 [12:12<24:24, 732.22s/it]

Epoch 1/3, Train Loss: 0.0402, Train Acc: 0.6144, Val Loss: 1.2053, Val Acc: 0.6982
Epoch: 1
-------
Looked at 0/180040 samples
Looked at 12800/180040 samples
Looked at 25600/180040 samples
Looked at 38400/180040 samples
Looked at 51200/180040 samples
Looked at 64000/180040 samples
Looked at 76800/180040 samples
Looked at 89600/180040 samples
Looked at 102400/180040 samples
Looked at 115200/180040 samples
Looked at 128000/180040 samples
Looked at 140800/180040 samples
Looked at 153600/180040 samples
Looked at 166400/180040 samples
Looked at 179200/180040 samples


 67%|██████▋   | 2/3 [26:15<13:17, 797.72s/it]

Epoch 2/3, Train Loss: 0.0366, Train Acc: 0.7315, Val Loss: 1.1730, Val Acc: 0.7310
Epoch: 2
-------
Looked at 0/180040 samples
Looked at 12800/180040 samples
Looked at 25600/180040 samples
Looked at 38400/180040 samples
Looked at 51200/180040 samples
Looked at 64000/180040 samples
Looked at 76800/180040 samples
Looked at 89600/180040 samples
Looked at 102400/180040 samples
Looked at 115200/180040 samples
Looked at 128000/180040 samples
Looked at 140800/180040 samples
Looked at 153600/180040 samples
Looked at 166400/180040 samples
Looked at 179200/180040 samples


100%|██████████| 3/3 [40:11<00:00, 803.83s/it]

Epoch 3/3, Train Loss: 0.0356, Train Acc: 0.7661, Val Loss: 1.1633, Val Acc: 0.7385


In [19]:
import torch
def save_model(model, target_dir, model_name):
    model_path = f"{target_dir}/{model_name}"
    print(f"[INFO] Saving model to: {model_path}")

    torch.save(obj=model.state_dict(),
             f=model_path)

In [22]:
save_model(model_1,"modules/models","model_1.pth")

[INFO] Saving model to: modules/models/model_1.pth
